In [31]:
#---
# Modules
#---
import numpy as np
import xarray as xr
import pandas as pd

from data import data_loader
from data import gesla_preprocessing
from data import era5_preprocessing

from gesla import GeslaDataset


In [32]:
#---
# Load Predictors
#----

# Select years and region
range_of_years = "1999-2008" 
subregion = "lon-0530_lat7040" 
season = "autumn" 
preprocess = "preprocess1"
predictor = "sp"

# Load daily mean data for all predictors
sp_dmean = data_loader.load_daymean_era5(
    range_of_years=range_of_years, 
    subregion=subregion, 
    season=season, 
    predictor=predictor,
    preprocess=preprocess,  
)

sp_dmean

<xarray.Dataset>
Dimensions:    (time: 910, bnds: 2, longitude: 141, latitude: 121)
Coordinates:
  * time       (time) datetime64[ns] 1999-09-01T11:30:00 ... 2008-11-30T11:30:00
  * longitude  (longitude) float32 -5.0 -4.75 -4.5 -4.25 ... 29.5 29.75 30.0
  * latitude   (latitude) float32 70.0 69.75 69.5 69.25 ... 40.5 40.25 40.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    sp         (time, latitude, longitude) float32 ...
Attributes:
    CDI:          Climate Data Interface version 1.9.3 (http://mpimet.mpg.de/...
    Conventions:  CF-1.6
    history:      Mon Mar 14 10:09:28 2022: cdo daymean sp_1999-2008_lon-0530...
    frequency:    day
    CDO:          Climate Data Operators version 1.9.4rc1 (http://mpimet.mpg....

In [66]:
#---
# Load Predictand
#---

# Create GESLA Dataset
meta_file = "resources/gesla/GESLA3_ALL.csv"
data_path = "resources/gesla/GESLA3.0_ALL.zip"

g3 = GeslaDataset(meta_file=meta_file, data_path=data_path)

# Select Stations
filenames = [
    'hanko-han-fin-cmems',
    # 'vahemadal-vah-est-cmems', just nan
    'pori-por-fin-cmems',
]

ds = g3.files_to_xarray(filenames)

# Select a season
if season == "autumn":
    get_season = gesla_preprocessing.is_autumn
elif season == "winter":
    get_season = gesla_preprocessing.is_winter

season_ds = ds.sel(date_time=get_season(ds['date_time.month']))

# Select only sea_level analysis data
df = gesla_preprocessing.get_analysis(season_ds)
df = df["sea_level"]

# Detrend data grouped by station
df_anom = gesla_preprocessing.detrend(df, level="station")

# Apply one hot encoding
df_isextreme = gesla_preprocessing.apply_dummies(df_anom, percentile=0.95, level="station")

# Convert to dataset
# nan values: no measurement at that timestamp for specific station
ds_extremes = df_isextreme.to_xarray()

ds_extremes

# # Select data of one station
# nstation = 0
# sea_level = ds_extremes.sel(station=nstation).values

<xarray.DataArray 'sea_level' (station: 2, date_time: 108811)>
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])
Coordinates:
  * station    (station) int64 0 1
  * date_time  (date_time) datetime64[ns] 1971-09-01 ... 2020-11-30T23:00:00

In [89]:
#---
# Predictor and predictand values of overlapping time series
#
# GESLA data is hourly. Needs to be daily, like ERA5. 
#---
predictor_time = pd.to_datetime(sp_dmean.time.values).date
predictand_time = pd.to_datetime(ds_extremes.date_time.values).date
sp = sp_dmean["sp"].values # Daily data
sl = ds_extremes.values # Hourly data

# Choose maximum per day, i.e. if one hour
# a day indicates an extreme surge, the whole day 
# is seen as extreme surge.
sl_dmax = []
for date in predictor_time:
    time_idx = np.where(predictand_time==date)[0] # Intersection of timeseries'
    slmax = np.max(sl[:, time_idx], axis=1)
    sl_dmax.append(slmax)

sl_dmax = np.array(sl_dmax)

#---
# Get only data without nan
#---
value_idx = np.where(~np.isnan(sl_dmax))
sl_dmax = sl_dmax[value_idx]

# value_idx = value_idx[0] # Time component
# predictor_time = predictor_time[value_idx] 
# sp = sp[value_idx, :, :]

In [75]:
x = sp
y = sl_dmax
t = predictor_time